In [2]:
from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import vision
import pandas as pd
import shutil 
import cv2
import os
import duckdb
import sqlalchemy
import random
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


#### inital variables for model name and visualisation

In [3]:

MODEL_PATH = "/home/abdullah/personal/github_projects/sm_city/result_renerator/model_arch_4.tflite"

_MARGIN = 10  
_ROW_SIZE = 10 
_FONT_SIZE = 1
_FONT_THICKNESS = 1
_TEXT_COLOR = (0, 255, 0)

In [4]:
base_options = core.BaseOptions(
      file_name=MODEL_PATH, use_coral=False, num_threads=4)
detection_options = processor.DetectionOptions(
      max_results=3, score_threshold=.4)   
options = vision.ObjectDetectorOptions(
      base_options=base_options, detection_options=detection_options)  
detector = vision.ObjectDetector.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Move test images from dataset/images to test_images , delete existing test_images folder if exists

In [ ]:

image_names = pd.read_csv('/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/test.csv')
image_names = image_names['image_path'].values
base_images = '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/images/'
image_names = [os.path.join(base_images, image_name) for image_name in image_names]
test_images_path = '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/result_renerator/test_images/'
if os.path.exists(test_images_path):
    shutil.rmtree(test_images_path)

os.makedirs(test_images_path)

for image_name in image_names:
    shutil.copy(image_name, test_images_path)

### List of all images in test_images folder 

In [ ]:
list_dist_dir = os.listdir(test_images_path)
list_dist_dir = [os.path.join(test_images_path, image_name) for image_name in list_dist_dir]


### Function for inference on test images

In [ ]:
def inference(image_array):
    input_tensor = vision.TensorImage.create_from_array(image_array)
    results = detector.detect(input_tensor)
    return results

#### distrubution of images vs classes

In [ ]:
%%sql 
select class , name , count(*) as total from '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/dataset/train.csv' 
group by class , name

#### class mapping

In [ ]:
class_map = {
    0: 'GRAFFITI',
    1: 'FADED_SIGNAGE',
    2: 'POTHOLES',
    3: 'GARBAGE',
    4: 'CONSTRUCTION_ROAD',
    5: 'BROKEN_SIGNAGE',
    6: 'BAD_STREETLIGHT',
    7: 'BAD_BILLBOARD',
    8: 'SAND_ON_ROAD',
    9: 'CLUTTER_SIDEWALK',
    10: 'UNKEPT_FACADE' 
}

#### Create a dataframe with columns as required by the submission format

In [ ]:
base_df = pd.DataFrame(columns=['class', 'image_path', 'name', 'xmax', 'xmin', 'ymax', 'ymin'])

def write_result(image, detection_result,image_name):
    global base_df
    if not detection_result.detections:
        base_df = base_df.append({'class': None,
                       'image_path': image_name,
                       'name': None,
                       'xmax': None,
                       'xmin': None,
                       'ymax': None,
                       'ymin': None}, ignore_index=True)
        return image


    for detection in detection_result.detections:
        bbox = detection.bounding_box
        start_point = bbox.origin_x, bbox.origin_y
        end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        cv2.rectangle(image, start_point, end_point, _TEXT_COLOR, 3)
        category = detection.categories[0]
        category_name = category.category_name
        probability = round(category.score, 2)
        result_text = category_name + ' (' + str(probability) + ')'
        text_location = (_MARGIN + bbox.origin_x,
                         _MARGIN + _ROW_SIZE + bbox.origin_y)
        cv2.putText(image, result_text, text_location, cv2.FONT_HERSHEY_PLAIN,
                    _FONT_SIZE, _TEXT_COLOR, _FONT_THICKNESS)
        print(category_name, probability)
        
        class_id = list(class_map.keys())[list(class_map.values()).index(category_name)]

        
        
        base_df = base_df.append({'class': class_id,
                       'image_path': image_name,
                       'name': category_name,
                       'xmax': bbox.origin_x + bbox.width,
                       'xmin': bbox.origin_x,
                       'ymax': bbox.origin_y + bbox.height,
                       'ymin': bbox.origin_y}, ignore_index=True)
    return image


#### Optional save images with drawn bounding boxes

In [ ]:
result_image_path = '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/result_renerator/result_images/'

In [ ]:
# for testing 
# random_img_list = random.sample(list_dist_dir, 10)

In [ ]:
for image_path in list_dist_dir:
    image = cv2.imread(image_path)
    results = inference(image)
    image_name = os.path.basename(image_path)
    image = write_result(image, results, image_name)
    # save image with drawed boxes
    



In [18]:
base_df.to_csv('result.csv', index=False)



In [24]:
%%sql 
-- Not sure why relative path doesn't work
-- using absolute path for now
create table result_data as select * from '/home/abdullah/personal/github_projects/efficientnet_lite_preprocess/result_renerator/result.csv' ; 


,Count
0,2422


In [25]:
%sql select count(*) from result_data where name not null

,count_star()
0,1127


In [32]:
%%sql
select name , count() as detections
 from result_data group by name order by count(*) desc

,name,detections
0,None,1295
1,CONSTRUCTION_ROAD,446
2,GARBAGE,241
3,POTHOLES,137
4,BAD_BILLBOARD,103
5,GRAFFITI,93
6,SAND_ON_ROAD,52
7,CLUTTER_SIDEWALK,29
8,BROKEN_SIGNAGE,18
9,UNKEPT_FACADE,7
